# Utilizing Machine Learning to Predict the Effects of COVID19 on Chicago Crime Rates

## Introduction 

A month and a half ago, the world ground to a halt due to COVID-19 and subsequent measures to curb its spread. Nonessential businesses worldwide were forced to close their doors and all individuals were told to engage in social distancing, minimizing contact with others and the outside world. 

Our group for this project consists of three biology majors and a sociology major. As biology students, the pandemic provides a uniquely relevant application of principles and concepts that we have covered in class. Meanwhile, from a sociological perspective, the halting of the economy and social distancing guidelines have affected many of the variables commonly used in models of crime rates, such as unemployment, walking traffic, and idleness. Based on our common interests, we investigated the use of machine learning technology in epidemiology and criminology and provided an example use case, examining the effect of COVID-19 on time, with Chicago Crime Rates (in particular domestic violence cases). 

Though the world’s focus has only recently turned towards predictive models of public health and the virus’ spread, the use of machine learning is not new to medical professionals and epidemiologists. Well before the onset of the coronavirus, companies and governments were using machine learning as one of many methods to attempt to detect epidemics and outbreaks in their infancy. This detection and tracking draw on a variety of sources. Data made available by hospitals and local government often show abnormal spikes in given diseases, perhaps suggesting an outbreak. Social media has proved to be a potent source of data for epidemiologists. Whereas data released by governments may be restricted, unreliable, or time delayed, social media posts are public, reliable (when aggregated), and instantaneous. In the case of diseases, whereas the first ‘official’ indication of an uptick in cases may come in a government’s weekly or monthly report, a bevy of signs may already be on social media, such as several individuals tweeting that they feel ill or otherwise overlooked local news outlets. Using natural language processing, machine learning programs are able detect signs of possible outbreaks and disease hotspots based on irregular upticks in suspicious posts or data. Using these methods, several data analytics companies were able to detect and predict the coronavirus outbreak, though not significantly before human experts. After an initial outbreak is detected, machine learning programs are used for predicting the extent of the spread, both in number of cases and geographical area. Machine learning, of course, relies on an abundance of data with which to train and evaluate a given model. A thoroughly documented pandemic on the scale of COVID-19 may lead to improved models. As the world economy moves to resume, managing local outbreaks will be imperative and the monitoring, detection, and analysis of real time data by machine learning programs may prove essential.

For more about machine learning and epidemiology, read one of these papers ([here][1] and [here][2]) or [this article][3] from Technology Review.


[1]:https://www.researchgate.net/publication/221876300_Machine_learning_and_data_mining_for_epidemic_surveillance/
[2]:https://aip.scitation.org/doi/pdf/10.1063/1.5005397/
[3]:https://www.technologyreview.com/2020/03/12/905352/ai-could-help-with-the-next-pandemicbut-not-with-this-one

## The Focus of our Data
The field of criminology is similarly rife with applications, both potential and realized, of machine learning technology. Much like epidemiologists, law enforcement can use machine learning programs to separate the noise inherent in social media data and identify suspicious behavior and predict crime. Machine learning can similarly be used to improve the accuracy of crime monitoring. In America, for example, gun violence is severely underreported, with fewer than one in four incidents of gun violence resulting in an emergency 911 call. Machine learning programs that use audio sensors and triangulation are beginning to be used for more accurate reporting of gunfire incidents. Such monitoring could also be used to identify areas for increased law enforcement presence or other aid, or in real time for more immediate arrival of law enforcement and emergency medical assistance. 

For more about criminology and machine learning, see these papers ([here][1] and [here][2]) and [this article][3] from the Brookings Institute.

[1]:https://ieeexplore.ieee.org/document/8614828
[2]:https://www.researchgate.net/publication/24101049_Identifying_the_Effect_of_Unemployment_on_Crime
[3]:https://www.brookings.edu/blog/up-front/2016/04/27/gun-violence-in-major-u-s-cities-is-massively-underreported/

## Coding for Machine Learning Model

When looking through possibilties of prediction models that best fit what we are looking for, we came across Google's Cloud AutoML Tables.
Cloud AutoML Tables were used in various types of prediction such as [retail stockout](https://aihub.cloud.google.com/p/products%2F094246f8-1aa2-4d2e-b8a6-5c65b7be01b9), [purchase](https://aihub.cloud.google.com/p/products%2Ffbc01462-221f-4e03-9285-900ba8bb386b), and even [music recommendations](https://aihub.cloud.google.com/p/products%2F9926b773-a357-4d97-ad4c-d54c273ed622). Due to AutoML Tables being so flexible and able to be accomodated for various types of datasets, we decided to try modeling our prediction with the AutoML Tables. 

In order to create the prediction model using Cloud AutoML Tables, we had to install and import the libraries.

In [20]:
# Install AutoML Tables client library
%pip install --quiet google-cloud-automl

Note: you may need to restart the kernel to use updated packages.


In [21]:
# AutoML library
from google.cloud import automl_v1beta1 as automl
import google.cloud.automl_v1beta1.proto.data_types_pb2 as data_types
import matplotlib.pyplot as plt

In [23]:
# Restarting the kernel to allow the automl_v1beta1 to be imported
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Creating API client to AutoML services

In [24]:
client = automl.AutoMlClient()
prediction_client = automl.PredictionServiceClient()

In [25]:
# Get the GCP location of our project.
location_path = client.location_path('qtm150spring20', 'us-central1')

### Creating dataset in AutoML Tables

In [26]:
# Create dataset
dataset_display_name = 'colab_trial1' 
create_dataset_response = client.create_dataset(location_path,{'display_name': dataset_display_name, 'tables_dataset_metadata': {}})
dataset_name = create_dataset_response.name
create_dataset_response

name: "projects/1079264031824/locations/us-central1/datasets/TBL6047631711628427264"
display_name: "colab_trial1"
create_time {
  seconds: 1587959417
  nanos: 358545000
}
etag: "AB3BwFpoDWMJs7CrZLldTsAOk2x6DYL3ffN_-SO_nt8XpsX-N5sZaCPB_q99u-34wrWG"
tables_dataset_metadata {
  stats_update_time {
  }
}

In [27]:
# Create bucket to store data in 
bucket_name = 'ml_project_bucket'

'ml_project_bucket'

### Importing Dependencies

In [28]:
!sudo pip install google-cloud-bigquery google-cloud-storage pandas pandas-gbq gcsfs oauth2client

import datetime
import pandas as pd

import gcsfs
from google.cloud import bigquery
from google.cloud import storage

client_bq = bigquery.Client(location='US', project='qtm150spring20')

### Importing training data

In [83]:
# As the data source is BigQuery
dataset_bq_input_uri = 'bq://bigquery-public-data.chicago_crime.crime'
# Define input configuration.
input_config = {'bigquery_source': {'input_uri': dataset_bq_input_uri}
}

In [85]:
# Import data
import_data_response = client.import_data(dataset_name, input_config)
print('Dataset import operation: {}'.format(import_data_response.operation))

Dataset import operation: name: "projects/1079264031824/locations/us-central1/operations/TBL4568883130269696000"
metadata {
  type_url: "type.googleapis.com/google.cloud.automl.v1beta1.OperationMetadata"
  value: "\032\014\010\377\331\231\365\005\020\260\272\217\316\002\"\014\010\377\331\231\365\005\020\260\272\217\316\002z\000"
}



In [93]:
# Checking to see if importing the data is complete 
# Continue with the rest only if this cell returns a `True`.
import_data_response.done()

True

### Reviewing the specs

In [200]:
# List table specs
list_table_specs_response = client.list_table_specs(dataset_name)
table_specs = [s for s in list_table_specs_response]
table_specs

[]

In [101]:
# Get column specs.
table_spec_name = table_specs[0].name
list_column_specs_response = client.list_column_specs(table_spec_name)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print column data types.
for column in column_specs: print(column, '-', column_specs[column].data_type)

unique_key - type_code: FLOAT64

case_number - type_code: CATEGORY

date - type_code: TIMESTAMP
time_format: "UNIX_MICROSECONDS"

block - type_code: STRING

iucr - type_code: FLOAT64
nullable: true

primary_type - type_code: STRING
nullable: true

description - type_code: STRING
nullable: true

location_description - type_code: STRING
nullable: true

arrest - type_code: CATEGORY

domestic - type_code: CATEGORY

beat - type_code: FLOAT64

district - type_code: FLOAT64
nullable: true

ward - type_code: FLOAT64
nullable: true

community_area - type_code: FLOAT64
nullable: true

fbi_code - type_code: FLOAT64
nullable: true

x_coordinate - type_code: FLOAT64
nullable: true

y_coordinate - type_code: FLOAT64
nullable: true

year - type_code: FLOAT64

updated_on - type_code: TIMESTAMP
time_format: "UNIX_MICROSECONDS"

latitude - type_code: FLOAT64
nullable: true

longitude - type_code: FLOAT64
nullable: true

location - type_code: STRING
nullable: true



### Updating columns by making them categorical

In [127]:
def create_update_column_sepc_dict(column_name, type_code, nullable): 
    update_column_spec_dict = {'name': column_specs[column_name].name, 'data_type': {'type_code': type_code,'nullable': nullable}} 
    return update_column_spec_dict

In [141]:
# Updating dataset
categorical_column_names = ['domestic', 'arrest', 'case_number','year', 'primary_type', 'description']
is_nullable = [True, True, False, True, True, True]

for i in range(len(categorical_column_names)): column_name = categorical_column_names[i] 
nullable = is_nullable[i]
update_column_spec_dict = create_update_column_sepc_dict(column_name, 'CATEGORY', nullable)
update_column_response = client.update_column_spec(update_column_spec_dict)

In [153]:
# Assigning a new label
label_column_name = 'domestic'
label_column_spec = column_specs[label_column_name]
label_column_id = label_column_spec.name.rsplit('/', 1)[-1]
print('Label column ID: {}'.format(label_column_id))

# Define the values of the fields to be updated.
update_dataset_dict = {'name': dataset_name,'tables_dataset_metadata': {'target_column_spec_id': label_column_id}
}

update_dataset_response = client.update_dataset(update_dataset_dict)
update_dataset_response

Label column ID: 5507973812529922048


name: "projects/1079264031824/locations/us-central1/datasets/TBL6047631711628427264"
display_name: "colab_trial1"
create_time {
  seconds: 1587959417
  nanos: 358545000
}
etag: "AB3BwFpu_nAROVv9ftHLFWK67q_YjCDz3ns51dO9kivyxi_bJYH9qF4OSOZvzOWPur87"
example_count: 7107081
tables_dataset_metadata {
  primary_table_spec_id: "9067074259943161856"
  target_column_spec_id: "5507973812529922048"
  stats_update_time {
    seconds: 1587966985
    nanos: 376000000
  }
}

## Creating a model

### Training the model

In [157]:
feature_list = list(column_specs.keys())
feature_list.remove('domestic')

model_display_name = 'domestic_model'
dataset_id = dataset_name.rsplit('/', 1)[-1]

model_dict = {'display_name': model_display_name,'dataset_id': dataset_id,'tables_model_metadata': {
    'target_column_spec': column_specs['domestic'],
    'input_feature_column_specs': [column_specs[f] for f in feature_list],
    'optimization_objective': 'MAXIMIZE_AU_PRC',
    'train_budget_milli_node_hours': 1000},}

create_model_response = client.create_model(location_path, model_dict)
print('Dataset import operation: {}'.format(create_model_response.operation))


Dataset import operation: name: "projects/1079264031824/locations/us-central1/operations/TBL429512122762788864"
metadata {
  type_url: "type.googleapis.com/google.cloud.automl.v1beta1.OperationMetadata"
  value: "\032\013\010\360\352\231\365\005\020\200\321\3170\"\013\010\360\352\231\365\005\020\200\321\3170R\000"
}



In [222]:
# Check if model training is complete
create_model_response.done()

True

## Evaluation of our prediction

In our dataset, for example, machine learning can be used to predict how domestic violence crimes are reported. We observe through our Machine Learning model, that many of the cases predicted “False”, were also actually deemed “False” (as it was not an actual domestic violence case). In the table below, we see that the prediction of domestic violence case, the prediction score (or strength of the prediction), as well as the actual reported cases outcome, correlate with each other. 

 <img src="https://github.com/jinahyejin/QTM250-example/blob/master/ML1.png?raw=true"
           alt="Data Analysis">
           
One aspect we do see with this model through this table, is that there is an underlying prediction modeling method through Record Count, that helps it gain better prediction of the outcomes of the cases reported. This however would also be an issue, as each case is it’s own independent report of domestic violence, illustrating the difficulty of creating assurance in our model to truly predict how a domestic violence case will be reported in the real world.

We started off with a binary classification model which was used to predict purchases. We used the chicago crime data that is found in BigQuery and had to clean and transform the data and save it separately in the google cloud storage bucket. 

 <img src="https://github.com/jinahyejin/QTM250-example/blob/master/ML2.png?raw=true"
           alt="Data Analysis">
           
The crimes data set has a very low percentage of values with a positive value of TRUE (the crime cases where it was a domestic violence). This led to a class imbalance problem shown in this table where we can see that even though the true label is positive, the prediction label shows as FALSE (negative) 83% of the time, illustrating a false negative. 
If we had the time to be able to modify and retrain the model, we could handle the imbalance by oversampling the positive class by random over sampling. This would mean there would be an artifical increase in the sessions with the positive label and thus would decrease the false negative.

This dataset had many columns with missing or invalid values which could not be used to add any signal to our model, so we had to drop them. These variables could have had an impact on the domestic variable in some way, but due to the model restraints, it would be difficult to predict how the variables would affect the model. 

## Rebuttal 

As a whole, the use of machine learning in criminology is not without controversy. Machine learning technologies with one example such as facial recognition, along with the prerequisite of increased surveillance, raises legitimate privacy concerns. Machine learning programs succeed with larger amounts and types of data. In the case of criminology, this often means the monitoring of law abiding citizens. Additionally, programs that seek to predict crime in certain individuals or locations have also been found to perpetuate and exacerbate bias. Even if the programming and data scientists are themselves without bias, the preexisting crime data used to train machine learning models may include profiling and bias.

For about issues of bias in machine learning applications to crime, see [this article][1] from Technology review.

The coronavirus has caused an unprecedented societal upheaval. Workers and students have been forced to stay home, businesses have closed and in many cases shut down, and unemployment has skyrocketed. The pandemic and the unique subsequent economic downturn have affected all aspects of life including several factors related to crime rates. For example, increased time shut away in homes has created concerns of increased domestic abuse and violence. Idleness has often been a suspected exacerbator of juvenile crime, a present danger given the closing of schools of all ages. Social distancing guidelines have reduced foot traffic, decreasing the number of watchful eyes and potential witnesses. Most importantly, poverty and unemployment, historically some of the surest predictors of crime, are expected to increase substantially during the coming depression.

[1]:https://www.technologyreview.com/2019/02/13/137444/predictive-policing-algorithms-ai-crime-dirty-data/

## Our data

Though models for crime during times of economic recession exist, the forced economic halt and continued social distancing guidelines make the present coming recession unique, and prior models that do not incorporate epidemiologic data may therefore be incomplete. To demonstrate the potential use of machine learning at the intersection of crime and epidemiology, we will explain what machine learning techniques and models we built, in order to explain at surface level the effects of COVID-19 on Chicago Domestic Violence Cases. 

After we have trained the model to attempt to predict variable outcomes without the Chicago Crime dataset, we wanted to observe the overall case reports of Domestic Violence. Domestic Violence is typically defined as violent or aggressive behavior within the home, and though that there could be effects on the numbers during the COVID-19 pandemic. But first, we would need to see the overall trends over time, and what the graph below illustrates is Domestic Violence cases reported, both True and False, since January 2001 until Today (April 26 2020).

<img src="https://github.com/jinahyejin/QTM250-example/blob/master/ML3.png?raw=true"
           alt="Data Analysis">
           
Although we do see an overall decrease in case counts over time in Chicago, we wanted to zoom in on the picture of our time-table today, during the COVID-19 outbreak. 

<img src="https://github.com/jinahyejin/QTM250-example/blob/master/ML4.png?raw=true"
           alt="Data Analysis">
           
What we see in this graph, is Domestic Violence reported cases since January of 2019. We begin in January 2019, because we wanted to see if another viral illness, in particular the Flu season of a typical year, may slow down domestic violence reports. We must keep in mind that the Flu season is not the only attribute of Chicago Winters, as the frigid cold may lead to many people staying inside, or less social interactions. On the basis of a Flu season (usually between January and February), we do see a slight drop in reported cases during this time. But fast forward a year into January 2020, where the COVID-19 pandemic had not reached Chicago then, but was just the beginning of the pandemic. We do see another overall decrease in the number of cases reported beginning between Dec 27, 2019 and February 25, 2020, until today, April 26, 2020. 

This evaluation is stunning in many fashions. The fact that the domestic violence cases could decrease over time could be due to external circumstances and overall happiness within society. But it also could be attributed to people not interacting with each other as much, much less so than during a typical Flu Season. But even after the deemed, “Flu Season”, through March and April, we see a continual decrease in the reported cases of Domestic Violence. Though we unfortunately cannot conclude that there truly are less Domestic Violence scenarios within the Chicago area, we can conclude from this graph that there are less reported cases, which may possibly equate to less destructive social interactions with one another. 

## Conclusion

In this project, we utilized machine learning to predict the effects of Covid-19 on Chicago Crime Rates, specifically focusing in on reports of domestic violence. There were some issues with training our model, with the class imbalance of having a much higher rate of "false" values than "true" values (cases that were labeled as domestic violence), which resulted in our prediction model having an 83% false negative rate. To avoid this issue in future studies, we recommend oversampling the positive class. 

With more people confined to their homes due to Illinois' stay-at-home order and closing of nonessential businesses, we hypothesized that the rate of domestic violence (which, by definition, happens within the home, often between family members or romantic partners) may increase with the spread of the novel coronavirus. However, our data showed otherwise. At least for reported cases of domestic violence, the rate has actually decreased within the Chicago area in the past few months (since February, through April). This trend could be influenced by a variety of confounding factors, and it is also worth noting that the time period from which the stay-at-home order was put in place (March 21st, 2020) til today is only a little over a month. It would be more advantageous to analyze the data for a longer period of time, which may very well be possible depending on how much longer this lockdown continues. It is difficult to determine a correlation with the limited data that we have. 

Nonetheless, machine learning is a useful resource in predicting the outcome of future cases, if executed correctly. As more data is collected in this unique and crucial time period, surely more empirical observations and discoveries will be made in the intersection of sociology and biology, as well as criminology and human health, which were our areas of interest. We hope to see similar studies which will help equip our society and leaders in better handling future epidemics and crises. 